In [1]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
from pprint import pprint
from time import sleep

# Google developer API key
from api_keys import gkey

In [2]:
# read zipcode file and create dataframe
zipcode_file = "../data/rawdata/Harris_County.csv"
lat_lng_file = "../data/rawdata/zipcode_lat_lngs.csv"
zip_codes = [77002, 77004, 77003, 77006, 77005, 77008, 77007, 77010, 
             77009, 77012, 77011, 77014, 77013, 77016, 77015, 77018, 
             77017, 77020, 77019, 77022, 77021, 77024, 77023, 77026, 
             77025, 77028, 77027, 77030, 77029, 77032, 77031, 77034, 
             77033, 77036, 77035, 77038, 77037, 77040, 77039, 77042, 
             77041, 77044, 77043, 77046, 77045, 77048, 77047, 77050, 
             77049, 77051, 77054, 77053, 77056, 77055, 77058, 77057, 
             77060, 77059, 77062, 77061, 77064, 77063, 77066, 77065, 
             77068, 77067, 77070, 77069, 77072, 77071, 77074, 77073, 
             77076, 77075, 77078, 77077, 77080, 77079, 77082, 77081, 
             77084, 77083, 77086, 77085, 77088, 77087, 77090, 77089, 
             77092, 77091, 77094, 77093, 77096, 77095, 77098, 77099, 
             77204, 77217, 77249, 77251, 77266, 77268, 77271, 77284, 
             77289, 77336, 77339, 77338, 77345, 77346, 77357, 77373, 
             77375, 77377, 77379, 77383, 77389, 77388, 77396, 77401, 
             77410, 77429, 77433, 77447, 77450, 77449, 77484, 77493, 
             77503, 77502, 77505, 77504, 77507, 77506, 77521, 77520, 
             77530, 77532, 77536, 77546, 77547, 77562, 77571, 77586, 77587, 77598]
lat_lng_df = pd.read_csv(lat_lng_file, encoding="ISO-8859-1")

#identify Harris zipcodes in lat_lng file
harris_df=pd.DataFrame()

i=0
for tzip in zip_codes:
    #harris_df.at[i,'zip code']=zip
    
    inner = lat_lng_df[lat_lng_df['ZIP']==tzip]
#     print(inner)
    harris_df = harris_df.append(inner)
    #harris_df.at[i,'lat']=harris_zip['LAT']
    i=i+1
harris_df.reset_index(inplace=True,drop=True)
harris_df.head()

,ZIP,LAT,LNG
0,77002,29.756845,-95.365652
1,77004,29.724893,-95.363752
2,77003,29.749778,-95.345885
3,77006,29.740970,-95.391301
4,77005,29.718435,-95.423555


In [3]:
ammenities_df = pd.DataFrame(data={"lat":"", "lng":"","# Restaurants":"", 'zipcode':'' }, index = [i for i in range(len(harris_df))])
ammenities_df.head()

,lat,lng,# Restaurants,zipcode
0,,,,
1,,,,
2,,,,
3,,,,
4,,,,


In [10]:
ammenities_df.iat[0, 2] = 5
ammenities_df.head()

,lat,lng,# Restaurants
0,29.7568,-95.3657,5
1,,,
2,,,
3,,,
4,,,


In [4]:
# base url and query specifics
ammenities_df = pd.DataFrame(data={"lat":"", "lng":"","# Restaurants":"", 'zipcode':''}, index = [i for i in range(len(harris_df))])
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 500
target_type = "bar"

# geocoordinates
#for i in range(0,len(harris_df),1):
for i in range(1):
    lat=harris_df.iloc[i,1]
    lng=harris_df.iloc[i,2]
    target_coordinates = f"{lat}, {lng}"
    print(target_coordinates)

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    #count number of ammenities
    places_data = response.json()
    
    pprint(places_data)


29.756845000000002, -95.365652
{'html_attributions': [],
 'next_page_token': 'CqQCGgEAAEdc-P6oXKFB-VTJy2qkksICijiBSMHiYE_5iMk96tDfk4483DTg44Tra0CJRYyxeNlwKGEAl0dcQ1BdkMCI78rMAkqnDP7hSMpVbfHuUu5ZG_gu1fe7UIfnwxMUih5fd2dVebLENLXv75Tsa7r8aZjcqytTp5eWW2bAspD46QwxyJAnbnam2VwE5hLwjuJeYvxLTLlMmdd9OgA2xRPNVaQRrYRBb9QSJgzq-0R-w9frCR7jOziP8bj63uamDns_u1uEkhmEYvV0GhIlcTPMV9AMu4PyIJmzeMT2fvp5LEj_CweJyKlSbYUm__D6KqmYwqjuYt0U9fB0EL-OKk-AWqTVlwHe6VjjWxlEZM1Vj5FJpqeXp5yCMfb3U_hwOwDpGRIQOCCj-0207vO3f10ehgUWuhoUvXzcWiu8QaSgFvbZ01zwThfWrKA',
 'results': [{'geometry': {'location': {'lat': 29.7536682,
                                        'lng': -95.36435349999999},
                           'viewport': {'northeast': {'lat': 29.7551393802915,
                                                      'lng': -95.36325116970849},
                                        'southwest': {'lat': 29.7524414197085,
                                                      'lng': -95.36594913029151}}},
              'icon':

In [5]:
page_token=places_data["next_page_token"]
print(page_token)
        
params = {
"key": gkey,
"pagetoken":page_token
}  

response = requests.get(base_url, params=params)
places_data = response.json()
pprint(places_data)

CqQCGgEAAEdc-P6oXKFB-VTJy2qkksICijiBSMHiYE_5iMk96tDfk4483DTg44Tra0CJRYyxeNlwKGEAl0dcQ1BdkMCI78rMAkqnDP7hSMpVbfHuUu5ZG_gu1fe7UIfnwxMUih5fd2dVebLENLXv75Tsa7r8aZjcqytTp5eWW2bAspD46QwxyJAnbnam2VwE5hLwjuJeYvxLTLlMmdd9OgA2xRPNVaQRrYRBb9QSJgzq-0R-w9frCR7jOziP8bj63uamDns_u1uEkhmEYvV0GhIlcTPMV9AMu4PyIJmzeMT2fvp5LEj_CweJyKlSbYUm__D6KqmYwqjuYt0U9fB0EL-OKk-AWqTVlwHe6VjjWxlEZM1Vj5FJpqeXp5yCMfb3U_hwOwDpGRIQOCCj-0207vO3f10ehgUWuhoUvXzcWiu8QaSgFvbZ01zwThfWrKA
{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 29.754811, 'lng': -95.3655795},
                           'viewport': {'northeast': {'lat': 29.7562761302915,
                                                      'lng': -95.36413751970849},
                                        'southwest': {'lat': 29.7535781697085,
                                                      'lng': -95.3668354802915}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/bar-71.png',
              'id': 'e505e8c9058f4e791

In [6]:
page_token=places_data["next_page_token"]
page_token

KeyError: 'next_page_token'

In [12]:
ammenities_df = pd.DataFrame(data={"lat":"", "lng":"","# Restaurants":"", 'zipcode':''}, index = [i for i in range(len(harris_df))])
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 1000
target_type = "restaurant"

# geocoordinates
#for i in range(0,len(harris_df),1):
for i in range(len(harris_df)):#len(harris_df)
    lat=harris_df.iloc[i,1]
    lng=harris_df.iloc[i,2]
    target_coordinates = f"{lat}, {lng}"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    #count number of ammenities
    places_data = response.json()
    places_data
    
    #count number of ammenities in first page
    places=places_data["results"]
    names = [place["name"] for place in places]
    ammenity_count_per_page=len(names)
    ammenity_count=len(names)
    
    #print(ammenity_count_per_page)
    
    #see if more than one page of results exists
    #while "next_page_token" in places_data:
    while "next_page_token" in places_data:
        sleep(3)
        page_token=places_data["next_page_token"]
        print(page_token)
        
        params = {
        "pagetoken":page_token,
        "key": gkey
        }  
        
        response = requests.get(base_url, params=params)
        places_data = response.json()
        #pprint(places_data)
        places=places_data["results"]
        names = [place["name"] for place in places]
        ammenity_count_per_page=0
        ammenity_count_per_page=len(names)
        print(ammenity_count_per_page)
        ammenity_count=ammenity_count+ammenity_count_per_page
    
    #populate dataframe
    ammenities_df['lat'].iloc[i]=lat
    ammenities_df['lng'].iloc[i]=lng
    ammenities_df['# Restaurants'].iloc[i]=ammenity_count
    ammenities_df.iat[i, 3] = harris_df.iloc[i, 0]
    ammenities_df
    
ammenities_df
i
#ammenity_count

CrQCIQEAALF7UZB8MF0yEOHpgnW1yD8EmcUdjkk0LqWKlzGaPoW-4XhpOy5_F23-iXV79iOXqoimENLUvDWatllvLiDkzdb0n5Bhofy1J83shFGV4diEmL4BKfD5BzaaHAPT_U3uVyYEnGUuxdu7H7cpJZERrbOYrDcxj89luR67bAO45J8dpIUh8mQRTUyie_dhMt_9Sm8gyO4SEsCx8hEBvUoXPN9KS6VUxsI262k6PMVAr0U20Og7ErApeSH8602MUxMsn93ypsfQcizu0xqVL_tVThiQlGxFGHu2eLZMfxY7uC_L51XEY8t0eEk-dlB0z9oEhaSZOTj0xmVPTiQTHz6E6Pk165NCmrl5yYUYQCxlFo-jnrtnI2BqZeE3LfHQmcr0Qr-IOOgisbIrdv9C-Svh6mISEJsPllFOZ9bzJ1nozJ6MgHAaFIDG-Hpo1wjHYRnpRQqtDwo37cMN
20
CtQDwQEAAKf13S1id7mtaOU45_1sOGcfr5EG3W0AnbWgq8mImz_PjumNHxqtfLlay5qV-IOKFD1-rexFSzXvalzmx_sC3f3zP7HEgC7Lb82SQihnjVt3qQMRkoEceuf1nUWIu3X9yJkq0sFqkCUwNbW0xyRYn_wR-mvUo518d6den8CTPjVBj83iVZMbpZC-3KsNZhjV53mvIUMNvzCbVOzTZo1zoaFnEGtsNn9WrXZ_BP8jf75XS-liuOoULYyyBuVetMljdehyZGYuX8RhMJHhOLPNfbf5WD9QxHPTazU9deTxQppeflCdOA69F3ZJFtgkTHhI-TrTiOSRxNbuIr8joh8H5UZ3rRcjfZVJQKFaORt6d7qe1Wtg8CKySrWTye-7mZ8-hzwI1XDU6lqKUSXbSO38vAT5GvioZq8iP4jLS4SXCorAZhn_EcnZoiBujYvNzEeFlX9vzv-W5jnV_rPA7Wr-uWGeiBkowg15sitnD4Vwzw5A1rEp5wqoYrGCnmtpLY1OwAdkpywi

134

In [238]:
zips=harris_df.loc[:,"ZIP"]
ammenities_with_zip=ammenities_df.join(zips)
ammenities_with_zip.sort_values(by="# Restaurants", ascending=False)

,lat,lng,# Restaurants,ZIP
0,29.7568,-95.3657,60,77002
43,29.7338,-95.4333,60,77046
94,29.7348,-95.4161,60,77098
3,29.741,-95.3913,60,77006
4,29.7184,-95.4236,60,77005
79,29.7121,-95.4809,60,77081
6,29.7715,-95.4111,60,77007
7,29.7536,-95.3598,60,77010
68,29.6997,-95.5848,60,77072
55,29.7441,-95.488,60,77057


In [226]:
ammenities_df

,lat,lng,# Restaurants
0,29.7568,-95.3657,60
1,29.7249,-95.3638,10
2,29.7498,-95.3459,41
3,29.741,-95.3913,60
4,29.7184,-95.4236,60
5,29.7982,-95.4169,29
6,29.7715,-95.4111,60
7,29.7536,-95.3598,60
8,29.7953,-95.3676,32
9,29.7185,-95.2741,11


In [14]:
#export data to csv
output_data_file = "../data/rawdata/restaurant_count_2.csv"
ammenities_df.to_csv(output_data_file, index=False, header=True)

In [22]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "error_message": "You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account",
    "html_attributions": [],
    "results": [],
    "status": "OVER_QUERY_LIMIT"
}


In [129]:
places=places_data["results"]
places
names = [place["name"] for place in places]
ammenity_count=len(names)
ammenity_count

10